In [5]:
!pip install ultralytics -q

# 2. GPU Kontrolü
import torch
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    print(f"✅ GPU Aktif: {gpu_name}")
    # T4 belleğini temizle
    torch.cuda.empty_cache()
else:
    raise Exception("❌ GPU bulunamadı! Runtime Type'ı 'T4 GPU' seçmelisin.")

✅ GPU Aktif: Tesla T4


In [6]:
import os
import shutil
import zipfile
import gdown

# --- AYARLAR ---
FILE_ID = '1TpBso-abIvZgnpUOocW-V-ZXUpMzuXXR' 
OUTPUT_ZIP = 'dental_ultimate.zip'
DATASET_DIR = '/content/dataset'

# İndirme
if not os.path.exists(OUTPUT_ZIP):
    print(f"⬇️ YOLO11 için veri seti indiriliyor...")
    gdown.download(id=FILE_ID, output=OUTPUT_ZIP, quiet=False, fuzzy=True)

# Çıkarma
if os.path.exists(OUTPUT_ZIP) and os.path.getsize(OUTPUT_ZIP) > 1000:
    print("📂 Zip açılıyor...")
    if os.path.exists(DATASET_DIR): shutil.rmtree(DATASET_DIR)
    
    with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref:
        zip_ref.extractall(DATASET_DIR)
        
    # YAML Güncelleme
    yaml_path = f"{DATASET_DIR}/processed_ultimate/data.yaml"
    new_yaml = f"""
train: {DATASET_DIR}/processed_ultimate/train/images
val: {DATASET_DIR}/processed_ultimate/val/images
nc: 4
names:
  0: Impacted
  1: Caries
  2: Periapical Lesion
  3: Deep Caries
"""
    with open(yaml_path, 'w') as f: f.write(new_yaml)
    print("✅ Veri ve Ayarlar Hazır.")
else:
    print("❌ İndirme Hatası. Drive izinlerini kontrol et.")

📂 Zip açılıyor...
✅ Veri ve Ayarlar Hazır.


In [7]:
from ultralytics import YOLO
from google.colab import drive
import os


# 1. MODELİ BAŞLAT (YOLO11 - Large)
model = YOLO('yolo11l.pt') 

print("🚀 YOLO11 ULTIMATE EĞİTİM")
print("Hedef: mAP > %70")

# 2. EĞİTİM
results = model.train(
    data='/content/dataset/processed_ultimate/data.yaml',
    project='/content/runs',
    name='v7_yolo11_large_1280',
    
    # --- Donanım ---
    epochs=100,
    imgsz=1280,           # Yüksek Çözünürlük
    batch=4,              
    device=0,
    workers=4,
    
    # --- Optimizasyon ---
    patience=20,
    optimizer='auto',     
    lr0=0.001,
    cos_lr=True,
    
    # --- Augmentation ---
    mosaic=1.0, 
    mixup=0.2,            
    copy_paste=0.2,       
    degrees=20.0,         
    translate=0.1,
    scale=0.7,            
    fliplr=0.5,
    
    save=True,
    exist_ok=True,
    verbose=True
)

🚀 YOLO11 ULTIMATE EĞİTİM
Hedef: mAP > %70
Ultralytics 8.3.233 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.2, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/dataset/processed_ultimate/data.yaml, degrees=20.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolo11l.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=v7_yolo11_large_1280, nbs=64, nms=False, opset=None, optimize=False, optimizer=au

In [8]:
import shutil
from google.colab import files
import os

# İndirilecek klasör
PROJECT_NAME = 'v7_yolo11_large_1280'
SOURCE_PATH = f'/content/runs/{PROJECT_NAME}'
OUTPUT_ZIP = f'{PROJECT_NAME}_results.zip'

if os.path.exists(SOURCE_PATH):
    print(f"📦 Sonuçlar paketleniyor...")
    shutil.make_archive(PROJECT_NAME + '_results', 'zip', SOURCE_PATH)
    
    print(f"⬇️ Bilgisayarına indiriliyor...")
    files.download(OUTPUT_ZIP)
else:
    print(f"❌ HATA: '{SOURCE_PATH}' bulunamadı. Eğitim henüz başlamamış olabilir.")

📦 Sonuçlar paketleniyor...
⬇️ Bilgisayarına indiriliyor...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

V8 modeli daha iyi mAP verdiği için bu model tercih edilmedi 